<a href="https://colab.research.google.com/github/kywch/geo-colab/blob/master/TreePedia_Step2_checking_gsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**To access the google streetview API, one should have the API key.**
* First, get started with Google Maps platform: https://developers.google.com/maps/gmp-get-started
* Then, get an API key to use Street View Static API: https://developers.google.com/maps/documentation/streetview/get-api-key

---
This notebook uses google-streetview package: https://pypi.org/project/google-streetview/. 
* For documentation, please see https://rrwen.github.io/google_streetview/



In [1]:
!pip install google-streetview

  Created wheel for google-streetview: filename=google_streetview-1.2.9-cp36-none-any.whl size=9781 sha256=72ac5ee2b60c69b89cd380e614bb3103d0e2ce71125a1713284581aa437d111e
  Stored in directory: /root/.cache/pip/wheels/e1/ce/c0/e90d7ee251659d215b7ab21a7cd80cfd65e1720fc3dca3b9ed
  Created wheel for kwconfig: filename=kwconfig-1.1.7-cp36-none-any.whl size=4981 sha256=29e9c140ff861c9ce647d024fba60f95753110dc9e1b5c642a0c07730a073ee4
  Stored in directory: /root/.cache/pip/wheels/22/fa/82/fed4c9e2829a6210470b121673adddedf169b6319112b3ba20
Successfully built google-streetview kwconfig


In [2]:
import pandas as pd
import google_streetview.api
import math

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls drive/My\ Drive/StreetPsych

Chicago  HydePark  key.txt


In [5]:
# Google Street View Static API key
with open('drive/My Drive/StreetPsych/key.txt') as file:
  api_key = file.read()

In [6]:
location = 'HydePark'
picked_df = pd.read_csv('drive/My Drive/StreetPsych/' + location + '/seed_' + location + '.csv')
picked_df

,Unnamed: 0,geometry,bearing,osm_index,lat,lng
0,0,POINT (-87.5816564665065 41.79278354249134),96.841944,0,41.792784,-87.581656
1,1,POINT (-87.58113712609745 41.79270690299883),103.889035,0,41.792707,-87.581137
2,2,POINT (-87.58426177854429 41.78864578433912),113.243840,4,41.788646,-87.584262
3,3,POINT (-87.58374468298702 41.78856196235375),96.709815,4,41.788562,-87.583745
4,4,POINT (-87.58322080593142 41.78852482917546),110.335822,4,41.788525,-87.583221
...,...,...,...,...,...,...
382,382,POINT (-87.60280457138116 41.79830587482549),358.951042,2043,41.798306,-87.602805
383,383,POINT (-87.60281746655603 41.79883090792616),358.951051,2043,41.798831,-87.602817
384,384,POINT (-87.60276990373274 41.79686538130811),358.980059,2044,41.796865,-87.602770
385,385,POINT (-87.60278244204305 41.79739042305226),358.980067,2044,41.797390,-87.602782


In [7]:
params = []
for row in picked_df.head(20).iterrows():
  params.append({
            'key': api_key,
            'location': str(row[1]['lat']) + ',' + str(row[1]['lng'])
  })
#'size': '600x480', # max 640x640 pixels
#print(params)

In [8]:
results = google_streetview.api.results(params)
#results.preview()
len(results.metadata)

20

In [9]:
# input - df: a Dataframe, chunkSize: the chunk size
# output - a list of DataFrame
# purpose - splits the DataFrame into smaller chunks
def split_dataframe(df, chunk_size = 200): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

def generate_param(df, api_key):
  params = []
  for row in df.iterrows():
    params.append({
              'key': api_key,
              'location': str(row[1]['lat']) + ',' + str(row[1]['lng'])
    })
  return params

In [ ]:
#split_dataframe(picked_df)[1][['lat','lng']].reset_index(drop=True)

In [12]:
results_df = pd.DataFrame()
split_dfs = split_dataframe(picked_df)
count = 1

for df in split_dfs:
  print('Processing ', count, ' / ', len(split_dfs))
  results = google_streetview.api.results(generate_param(df, api_key))
  gsv_info = []
  for row in results.metadata:
    if 'OK' in row['status']:
      if 'Google' in row['copyright']:
        gsv_info.append([row['location']['lat'], row['location']['lng'], row['pano_id'], row['date'], row['status']])
      else:
        gsv_info.append([math.nan, math.nan, math.nan, math.nan, math.nan])
    else:
      gsv_info.append([math.nan, math.nan, math.nan, math.nan, math.nan])

  results_df = results_df.append( pd.DataFrame(gsv_info, columns = ['gsv_lat', 'gsv_lng', 'pano_id', 'date', 'status']) )
  count += 1

print('Done.')  


Processing  1  /  2
Processing  2  /  2
Done.


In [28]:
combined_df = picked_df[['lat','lng','bearing','osm_index']].merge(results_df.reset_index(drop=True), left_index=True, right_index=True, suffixes=(False, False))
combined_df  

,lat,lng,bearing,osm_index,gsv_lat,gsv_lng,pano_id,date,status
0,41.792784,-87.581656,96.841944,0,41.792797,-87.581747,GHRFbz2JAAvkDmWHU7oswA,2011-06,OK
1,41.792707,-87.581137,103.889035,0,41.792710,-87.581160,ufrhNaqGu6SQVbLVgT_w_w,2011-06,OK
2,41.788646,-87.584262,113.243840,4,41.788594,-87.584125,wEtBw3PTguAUA2oyo0hs-Q,2011-06,OK
3,41.788562,-87.583745,96.709815,4,41.788553,-87.583791,zMSXwtuYNfe_W-N5telkzA,2011-06,OK
4,41.788525,-87.583221,110.335822,4,41.788504,-87.583216,aLoxEIp1Jaw6-i7KNE74Lw,2011-06,OK
...,...,...,...,...,...,...,...,...,...
382,41.798306,-87.602805,358.951042,2043,41.798292,-87.602808,EUabOjfF7IvKIRLocsyfyA,2019-05,OK
383,41.798831,-87.602817,358.951051,2043,41.798839,-87.602822,cxYcRxH5dSerNG_3ExXstg,2019-05,OK
384,41.796865,-87.602770,358.980059,2044,41.796822,-87.602765,7LyyRE6hKmFNrtjI5EagDQ,2019-04,OK
385,41.797390,-87.602782,358.980067,2044,41.797362,-87.602782,BhgMwSc2pJNzR3W9XNcDjQ,2019-04,OK


In [29]:
combined_df.to_csv('drive/My Drive/StreetPsych/' + location + '/gsv_seed_' + location + '.csv')